# Final Project
by Malte Jörg

---
## 1. Bussiness Problem
- Compare Toronto's and New York's Nightlife
- Bars, Cafés and Restaurants

---
## 2. Data description
- get neighborhood data from the cities
- get venue data via foursquare

Python library imports:

In [1]:
import pandas as pd
import numpy as np
import geocoder
import json
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
from sklearn.cluster import KMeans

---
## 3. Data gathering
- TBD

### a) New York Dataset

In [2]:
# open new york dataset from downloaded json file
with open('data/newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [3]:
# only use data from features key
ny_data = newyork_data['features']

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
df_ny = pd.DataFrame(columns=column_names)

# put relevant data from features into dataframe
for data in ny_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    df_ny = df_ny.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
print('Shape of Dataframe: ', df_ny.shape)
df_ny.head()

Shape of Dataframe:  (306, 4)


,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [4]:
print('Unique Neighborhoods in New York: ', len(df_ny['Neighborhood'].unique()))
df_ny.groupby('Borough').count()

Unique Neighborhoods in New York:  302


,Neighborhood,Latitude,Longitude
Borough,,,
Bronx,52,52,52
Brooklyn,70,70,70
Manhattan,40,40,40
Queens,81,81,81
Staten Island,63,63,63


*The New York Dataframe consists of 306 geographical coordinates from 302 unique neighborhoods in 5 boroughs.*

### b) Toronto Dataset

In [5]:
# scraping table from wikipedia url
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_tor = pd.read_html(url)[0]

In [6]:
# drop not assigned boroughs and rename neighborhoods
df_tor = df_tor[~df_tor['Borough'].isin(['Not assigned'])].reset_index(drop=True)
for i in range(len(df_tor.index)):
    if df_tor['Neighborhood'].loc[i] == 'Not assigned':
       df_tor['Neighborhood'].loc[i] = df_tor['Borough'].loc[i]
    else:
        pass

In [7]:
# get geospatial coordinates from downloaded csv-file and assign to toronto dataframe
df_tor_geo = pd.read_csv('data/Geospatial_Coordinates.csv')
latitude = []
longitude = []
for postal_code in df_tor['Postal Code']:
    i = list(df_tor_geo['Postal Code']).index(postal_code)
    latitude.append(df_tor_geo['Latitude'].loc[i])
    longitude.append(df_tor_geo['Longitude'].loc[i])  
df_tor['Latitude'] = latitude
df_tor['Longitude'] = longitude

In [8]:
# drop postal code column to get consistent dataframe between new york and toronto
df_tor.drop(['Postal Code'], axis=1, inplace=True)

In [9]:
print('Shape of Dataframe: ', df_tor.shape)
df_tor.head()

Shape of Dataframe:  (103, 4)


,Borough,Neighborhood,Latitude,Longitude
0,North York,Parkwoods,43.753259,-79.329656
1,North York,Victoria Village,43.725882,-79.315572
2,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [10]:
print('Unique Neighborhoods in Toronto: ', len(df_tor['Neighborhood'].unique()))
df_tor.groupby('Borough').count()

Unique Neighborhoods in Toronto:  99


,Neighborhood,Latitude,Longitude
Borough,,,
Central Toronto,9,9,9
Downtown Toronto,19,19,19
East Toronto,5,5,5
East York,5,5,5
Etobicoke,12,12,12
Mississauga,1,1,1
North York,24,24,24
Scarborough,17,17,17
West Toronto,6,6,6


*The Toronto Dataframe consists of 103 geographical coordinates from 99 unique neighborhoods in 10 boroughs.*

### c) Examine geographical data in datasets

In [11]:
toronto_lat = 43.651070
toronto_long = -79.347015
map_toronto = folium.Map(location=[toronto_lat, toronto_long], zoom_start=10)
for lat, lng, borough, neighborhoods in zip(df_tor['Latitude'], df_tor['Longitude'], df_tor['Borough'], df_tor['Neighborhood']):
    label = '{}, {}'.format(borough, neighborhoods)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
map_toronto

In [12]:
newyork_lat = 40.730610
newyork_long = -73.935242
map_newyork = folium.Map(location=[newyork_lat, newyork_long], zoom_start=10)
for lat, lng, borough, neighborhoods in zip(df_ny['Latitude'], df_ny['Longitude'], df_ny['Borough'], df_ny['Neighborhood']):
    label = '{}, {}'.format(borough, neighborhoods)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)
map_newyork 

*As you can see already, the new york dataset is much bigger and dense than the Toronto dataset. This points out that a relative analysis of the venues is needed, to get a good comparison of the two cities and the consisting neighborhoods and boroughs.*

In the next step of the data gathering step I will create two dataframes which consist popular venues in the two cities.

### d) New York venue gathering

In [13]:
# foursquare api details:
CLIENT_ID = '22T4CBV0WIVKXKQERQRP05B0Q2J534DO4MQHL54U3J0MPMMA'
CLIENT_SECRET = '1ODAI15QLXWHQDRJQDEDPNECDMUJZQNSJRB3SLHLHKVUQUAS'
VERSION = '20180605'
RADIUS = 500
LIMIT = 100

In [21]:
# function to get venues from foursquare by geographical coordinates
def getVenues(borough_names, neighborhood_names, latitudes, longitudes, RADIUS, LIMIT): 
    venues_list=[]

    for bname, nname, lat, lng in zip(borough_names, neighborhood_names, latitudes, longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                lat, 
                lng, 
                RADIUS, 
                LIMIT)
        
        results = requests.get(url).json()["response"]['groups'][0]['items']  # foursquare request            
        venues_list.append([(
            bname,
            nname, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['shortName']) for v in results])

    venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    venues.columns = ['Borough',
                  'Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    print('done')    
    return(venues)

In [22]:
exec_fsq = False    # execute foursquare api venue exploration

In [19]:
# get new york venues
if exec_fsq == True:
    ny_venues = getVenues(borough_names=df_ny['Borough'],
                                neighborhood_names=df_ny['Neighborhood'],
                                latitudes=df_ny['Latitude'],
                                longitudes=df_ny['Longitude'],
                                RADIUS=RADIUS,
                                LIMIT=LIMIT
                                )

done


In [48]:
# get new york venues
if exec_fsq == True:
    tor_venues = getVenues(borough_names=df_tor['Borough'],
                                neighborhood_names=df_tor['Neighborhood'],
                                latitudes=df_tor['Latitude'],
                                longitudes=df_tor['Longitude'],
                                RADIUS=RADIUS,
                                LIMIT=LIMIT
                                )

done


In [72]:
# save or read dataframes for later analysis
if exec_fsq == True:
    tor_venues.to_csv('data/toronto_venues.csv')
    ny_venues.to_csv('data/newyork_venues.csv')
else:
    tor_venues = pd.read_csv('data/toronto_venues.csv', index_col=0)
    ny_venues = pd.read_csv('data/newyork_venues.csv', index_col=0)
print('Shape of Toronto venues Dataframe: ', tor_venues.shape)
print('Shape of New York venues Dataframe: ', ny_venues.shape)


Shape of Toronto venues Dataframe:  (2130, 8)
Shape of New York venues Dataframe:  (9983, 8)


*This concludes the data gathering step of this project. There are 2130 venues in the Toronto dataset and 9983 venues in the New York dataset. In the following step these datasets has to be preprocessed to prepare the date for further analysis.*

---
## 4. Data Preprocessing
- Examination of the datasets
- Deleting multiple venues in the dataset
- Merge similar venue categories

### a) Examination

In [32]:
tor_venues.head()

,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,North York,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,North York,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink
2,North York,Parkwoods,43.753259,-79.329656,Corrosion Service Company Limited,43.752432,-79.334661,Construction
3,North York,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey
4,North York,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese


In [37]:
print('Total amount of venues in the dataset: ', len(tor_venues.index))
print('Total amount of unique venues in the dataset: ', len(tor_venues['Venue'].unique()))

Total amount of venues in the dataset:  2130
Total amount of unique venues in the dataset:  1396


In [38]:
print('Total amount of unique venue categories in the dataset: ', len(tor_venues['Venue Category'].unique()))
print('List of unqiue venue Categories:')
tor_venues['Venue Category'].unique()

Total amount of unique venue categories in the dataset:  268
List of unqiue venue Categories:


array(['Park', 'Food & Drink', 'Construction', 'Hockey', 'Portuguese',
       'Coffee Shop', 'Intersection', 'Pizza', 'Bakery', 'Distributor',
       'Spa', 'Pub', 'Restaurant', 'Breakfast', 'Gym / Fitness',
       'Historic Site', "Farmer's Market", 'Performing Arts',
       'Chocolate Shop', 'Desserts', 'French', 'Café', 'Yoga Studio',
       'Theater', 'Event Space', 'Shoes', 'Ice Cream', 'Art Gallery',
       'Cosmetics', 'Asian', 'Electronics', 'Bank', 'Beer Store',
       'Antiques', 'Boutique', 'Furniture / Home', 'Vietnamese',
       'Apparel', 'Accessories', "Women's Store", 'Shop', 'Arts & Crafts',
       'Athletics & Sports', 'Italian', 'Sushi', 'Creperie', 'Beer Bar',
       'Burritos', 'Mexican', 'Hobbies', 'Diner', 'Fried Chicken',
       'Discount Store', 'Smoothie Shop', 'Sandwiches', 'Gym', 'Bar',
       'Auditorium', 'Sculpture', 'Music Venue', 'Fast Food', 'Caribbean',
       'Japanese', 'Gastropub', 'Pharmacy', 'Pet Store', 'Comic Shop',
       'Tea Room', 'Plaza', 

In [40]:
ny_venues.head()

,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bronx,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Desserts
1,Bronx,Wakefield,40.894705,-73.847201,Walgreens,40.896528,-73.844700,Pharmacy
2,Bronx,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream
3,Bronx,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
4,Bronx,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donuts


In [41]:
print('Total amount of venues in the dataset: ', len(ny_venues.index))
print('Total amount of unique venues in the dataset: ', len(ny_venues['Venue'].unique()))

Total amount of venues in the dataset:  9983
Total amount of unique venues in the dataset:  7711


In [42]:
print('Total amount of unique venue categories in the dataset: ', len(ny_venues['Venue Category'].unique()))
print('List of unqiue venue Categories:')
ny_venues['Venue Category'].unique()

Total amount of unique venue categories in the dataset:  422
List of unqiue venue Categories:


array(['Desserts', 'Pharmacy', 'Ice Cream', 'Donuts', 'Gas Station',
       'Sandwiches', 'Food', 'Pizza', 'Laundromat', 'Discount Store',
       'Post Office', 'Bagels', 'Grocery Store', 'Fast Food',
       'Restaurant', 'Bus Station', 'Baseball Field', 'Deli / Bodega',
       'Basketball Court', 'Park', 'Caribbean', 'Diner', 'Seafood',
       'Bowling Alley', 'Automotive', 'Food & Drink', 'Platform',
       'Convenience Store', 'Bus Stop', 'Plaza', 'River', 'Bank',
       'Food Truck', 'Gym', 'Gourmet', 'Latin American', 'Pub', 'Burgers',
       'Beer Bar', 'Warehouse Store', 'Coffee Shop', 'Spanish', 'Mexican',
       'Bar', 'Wings', 'Thrift / Vintage', 'Trail', 'Supermarket',
       'Bakery', 'Candy Store', 'Café', 'Rental Car', 'Fried Chicken',
       'Breakfast', 'Mattress Store', 'Pet Store', 'Supplement Shop',
       'Shipping Store', 'Sports Bar', 'Furniture / Home', 'Liquor Store',
       'Chinese', 'Outdoor Sculpture', 'Bus', 'Yoga Studio', 'Tennis',
       'Department Store

*The Examination of the datasets shows a vast diversity of the venue categories. To get a better understanding of the boroughs in Toronto and New York, a more clear categorization of the venues is decisive.*

### b) Drop multiple venues

In [71]:
# drop multiple venues in dataset by venue coordinates, name and borough
def dropVenues(df):
    del_list = []
    for i in range(len(df.index)):
        coor = [df['Venue Latitude'][i], df['Venue Longitude'][i]]
        venue = df['Venue'][i]
        bor = df['Borough'][i]
        for v in range(i+1,len(df.index)):
            if df['Venue Latitude'][v] == coor[0] and df['Venue Longitude'][v] == coor[1] and df['Venue'][v] == venue and df['Borough'][v] == bor:
                del_list.append(v)
    
    df.drop(del_list, inplace=True)
    return(df.reset_index(drop=True))

In [73]:
tor_venues = dropVenues(tor_venues)
print('Shape of Toronto venues Dataframe: ', tor_venues.shape)

Shape of Toronto venues Dataframe:  (1706, 8)


In [74]:
ny_venues = dropVenues(ny_venues)
print('Shape of New York venues Dataframe: ', ny_venues.shape)

Shape of New York venues Dataframe:  (9434, 8)


In [77]:
exec_drop = True

In [78]:
# save or read dataframes for later analysis
if exec_drop == True:
    tor_venues.to_csv('data/toronto_venues_drop.csv')
    ny_venues.to_csv('data/newyork_venues_drop.csv')
else:
    tor_venues = pd.read_csv('data/toronto_venues_drop.csv', index_col=0)
    ny_venues = pd.read_csv('data/newyork_venues_drop.csv', index_col=0)

*The amount of venues that got drop from the datasets is lower than the amount of unqiue venues. This is possible because a venues could possibly exist in multiple boroughs, because of the radius assigned to the foursquare exploration. The further analysis is continued with this dataset and it is assumed that these venues consist to each of these boroughs.*

### c) Merge venue categories

In [83]:
cat_url = 'https://api.foursquare.com/v2/venues/categories?&client_id={}&client_secret={}&v={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION)
cat_request = requests.get(cat_url).json()

In [105]:
main = []
cate = []
for main_cat in cat_request['response']['categories']:
    for sec_cat in main_cat['categories']:
        main.append(main_cat['name'])
        cate.append(sec_cat['name'])
        for thi_cat in sec_cat['categories']:
            main.append(main_cat['name'])
            cate.append(thi_cat['name'])
            for fou_cat in thi_cat['categories']:
                main.append(main_cat['name'])
                cate.append(fou_cat['name'])
                for fit_cat in fou_cat['categories']:
                    main.append(main_cat['name'])
                    cate.append(fit_cat['name'])
categories = pd.DataFrame()
categories['Main Category'] = main
categories['Category'] = cate
categories.head()

,Main Category,Category
0,Arts & Entertainment,Amphitheater
1,Arts & Entertainment,Aquarium
2,Arts & Entertainment,Arcade
3,Arts & Entertainment,Art Gallery
4,Arts & Entertainment,Bowling Alley


In [106]:
def addMainCategory(df):
    for i in range(len(df.index)):
        

(933, 2)